In [32]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target'].astype(int)


subset_size = 150
X, _, y, _ = train_test_split(X, y, train_size=subset_size, random_state=42, stratify=y)


pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)


def feature_map(circuit, data_point):
    for i in range(len(data_point)):
        circuit.h(i)
    for i, value in enumerate(data_point):
        circuit.rx(2 * np.pi * float(value), i)
        circuit.ry(2 * np.pi * float(value), i)
    return circuit


def augmented_data(data_point, samples=3):
    circuits = []
    base_circuit = QuantumCircuit(len(data_point))
    feature_map(base_circuit, data_point)
    circuits.append(base_circuit)

    for _ in range(samples - 1):
        circuit = QuantumCircuit(len(data_point))
        feature_map(circuit, data_point)
        for qubit in range(len(data_point)):
            circuit.rx(np.random.uniform(-0.2, 0.2), qubit)
            circuit.ry(np.random.uniform(-0.2, 0.2), qubit)
            circuit.rz(np.random.uniform(-0.2, 0.2), qubit)
        if len(data_point) > 1:
            circuit.cx(0, 1)
        if len(data_point) > 2:
            circuit.cx(1, 2)
        circuits.append(circuit)
    return circuits


def simulation(circuits):
    simulator = Aer.get_backend('statevector_simulator')
    results = []
    for circuit in circuits:
        transpiled = transpile(circuit, simulator)
        job = simulator.run(transpiled)
        result = job.result()
        statevector = result.get_statevector()
        results.append(statevector)
    return results


arg_set = []
for data_point in X_pca:
    circuits = augmented_data(data_point, samples=3)
    results = simulation(circuits)
    arg_set.append(results)


arg_features = []
for result_set in arg_set:
    for statevector in result_set:
        probabilities = np.abs(statevector) ** 2
        arg_features.append(probabilities)

arg_features = np.array(arg_features)


arg_labels = np.repeat(y, 3)


X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

svm_orig = SVC(kernel='linear')
svm_orig.fit(X_train_orig, y_train_orig)
y_pred_orig = svm_orig.predict(X_test_orig)

print("\n=== Classical SVM on Original PCA-Reduced Data ===")
print(classification_report(y_test_orig, y_pred_orig))
print("Accuracy:", accuracy_score(y_test_orig, y_pred_orig))


X_arg_train, X_arg_test, y_arg_train, y_arg_test = train_test_split(
    arg_features, arg_labels, test_size=0.2, random_state=42, stratify=arg_labels
)

svm_aug = SVC(kernel='rbf')  
svm_aug.fit(X_arg_train, y_arg_train)
y_pred_aug = svm_aug.predict(X_arg_test)

print("\n=== Classical SVM on Quantum-Augmented Data ===")
print(classification_report(y_arg_test, y_pred_aug))
print("Accuracy:", accuracy_score(y_arg_test, y_pred_aug))



=== Classical SVM on Original PCA-Reduced Data ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3
           2       1.00      0.33      0.50         3
           3       0.00      0.00      0.00         3
           4       1.00      0.67      0.80         3
           5       0.20      0.33      0.25         3
           6       0.50      0.67      0.57         3
           7       0.33      0.33      0.33         3
           8       0.75      1.00      0.86         3
           9       0.50      0.67      0.57         3

    accuracy                           0.60        30
   macro avg       0.63      0.60      0.59        30
weighted avg       0.63      0.60      0.59        30

Accuracy: 0.6

=== Classical SVM on Quantum-Augmented Data ===
              precision    recall  f1-score   support

           0       0.73      0.89      0.80         9
           1     